In [22]:
import requests

# Define the API endpoint
api_url = "https://prim.iledefrance-mobilites.fr/marketplace/stop-monitoring"

# API key (access token)
api_key = "bH6c8yOh6cFDhml3QJ7eW0KDBSQkzzef"  # Token from token_ratp.txt

# Monitoring references for each stop
monitoring_refs = {
    "144": "STIF:StopPoint:Q:413090:",  # StopPoint for bus 144
    "244": "STIF:StopPoint:Q:421321:"   # StopPoint for bus 244
}

# Bus line identifiers for 144 and 244
bus_lines = {
    "144": "STIF:Line::C01169:",  # LineRef for bus 144
    "244": "STIF:Line::C01240:"   # LineRef for bus 244
}

# Headers for the request
headers = {
    "Accept": "application/json",
    "apikey": api_key
}

# Function to query the API for a specific bus line and monitoring reference (stop point)
def get_next_passages(line_ref, monitoring_ref):
    params = {
        "MonitoringRef": monitoring_ref,
        "LineRef": line_ref
    }
    response = requests.get(api_url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# Fetch the next passages for each bus line and stop point pair
results = []
for bus, line_ref in bus_lines.items():
    monitoring_ref = monitoring_refs[bus]  # Get the corresponding MonitoringRef
    print(f"Fetching next passages for bus {bus} at stop {monitoring_ref}...")
    result = get_next_passages(line_ref, monitoring_ref)
    if result:
        print(result)  # Print the API response (you can format this as needed)
        results.append(result)


Fetching next passages for bus 144 at stop STIF:StopPoint:Q:413090:...
{'Siri': {'ServiceDelivery': {'ResponseTimestamp': '2024-09-23T22:07:44.770Z', 'ProducerRef': 'IVTR_HET', 'ResponseMessageIdentifier': 'IVTR_HET:ResponseMessage:6be0c031-9027-4ea0-985e-641b4732a597:LOC:', 'StopMonitoringDelivery': [{'ResponseTimestamp': '2024-09-23T22:07:44.785Z', 'Version': '2.0', 'Status': 'true', 'MonitoredStopVisit': []}]}}}
Fetching next passages for bus 244 at stop STIF:StopPoint:Q:421321:...
{'Siri': {'ServiceDelivery': {'ResponseTimestamp': '2024-09-23T22:07:45.108Z', 'ProducerRef': 'IVTR_HET', 'ResponseMessageIdentifier': 'IVTR_HET:ResponseMessage:fee3f3ad-33af-4fbd-a69b-6e48ffbd4212:LOC:', 'StopMonitoringDelivery': [{'ResponseTimestamp': '2024-09-23T22:07:45.111Z', 'Version': '2.0', 'Status': 'true', 'MonitoredStopVisit': [{'RecordedAtTime': '2024-09-23T21:41:17.346Z', 'ItemIdentifier': 'RATP-SIV:Item::20240923.68.R.C01240.PALS.IDFM.C01240.R.RATP.64936:LOC', 'MonitoringRef': {'value': 'STI

In [23]:
results

[{'Siri': {'ServiceDelivery': {'ResponseTimestamp': '2024-09-23T22:07:44.770Z',
    'ProducerRef': 'IVTR_HET',
    'ResponseMessageIdentifier': 'IVTR_HET:ResponseMessage:6be0c031-9027-4ea0-985e-641b4732a597:LOC:',
    'StopMonitoringDelivery': [{'ResponseTimestamp': '2024-09-23T22:07:44.785Z',
      'Version': '2.0',
      'Status': 'true',
      'MonitoredStopVisit': []}]}}},
 {'Siri': {'ServiceDelivery': {'ResponseTimestamp': '2024-09-23T22:07:45.108Z',
    'ProducerRef': 'IVTR_HET',
    'ResponseMessageIdentifier': 'IVTR_HET:ResponseMessage:fee3f3ad-33af-4fbd-a69b-6e48ffbd4212:LOC:',
    'StopMonitoringDelivery': [{'ResponseTimestamp': '2024-09-23T22:07:45.111Z',
      'Version': '2.0',
      'Status': 'true',
      'MonitoredStopVisit': [{'RecordedAtTime': '2024-09-23T21:41:17.346Z',
        'ItemIdentifier': 'RATP-SIV:Item::20240923.68.R.C01240.PALS.IDFM.C01240.R.RATP.64936:LOC',
        'MonitoringRef': {'value': 'STIF:StopPoint:Q:421321:'},
        'MonitoredVehicleJourney': {'L

In [16]:
import pandas as pd

data = result

# Function to process the API response and extract relevant information into a DataFrame
def process_response(response, bus_number):
    monitored_visits = response["Siri"]["ServiceDelivery"]["StopMonitoringDelivery"][0]["MonitoredStopVisit"]

    data = []
    for visit in monitored_visits:
        journey = visit["MonitoredVehicleJourney"]
        direction = journey["DirectionName"][0]["value"]
        destination = journey["DestinationName"][0]["value"]
        stop_name = journey["MonitoredCall"]["StopPointName"][0]["value"]
        expected_time = journey["MonitoredCall"]["ExpectedDepartureTime"]
        status = journey["MonitoredCall"]["DepartureStatus"]

        data.append({
            "Bus": bus_number,
            "Direction": direction,
            "Destination": destination,
            "StopPointName": stop_name,
            "ExpectedDepartureTime": expected_time,
            "DepartureStatus": status
        })

    return pd.DataFrame(data)

# Process both bus responses and combine them into a single DataFrame
df_144 = process_response(results[0], "144")
df_244 = process_response(results[1], "244")

# Combine the results for both buses
df_combined = pd.concat([df_144, df_244], ignore_index=True)


In [25]:
df_combined

,Bus,Direction,Destination,StopPointName,ExpectedDepartureTime,DepartureStatus
0,244,André Maurois / Porte maillot,Porte Maillot,Rueil-Malmaison RER,2024-09-23T22:05:00.000Z,onTime
1,244,André Maurois / Porte maillot,Porte Maillot,Rueil-Malmaison RER,2024-09-23T22:30:00.000Z,onTime


### TODO: add this to understand the API timestamp in local time

In [24]:
from datetime import datetime
import pytz

# Function to convert UTC time to Paris (GMT+1)
def convert_to_paris_time(utc_time_str):
    # Parse the UTC time string
    utc_time = datetime.strptime(utc_time_str, '%Y-%m-%dT%H:%M:%S.%fZ')

    # Set the time zone for UTC
    utc_zone = pytz.timezone('UTC')
    utc_time = utc_zone.localize(utc_time)

    # Convert to Paris time
    paris_zone = pytz.timezone('Europe/Paris')
    paris_time = utc_time.astimezone(paris_zone)

    return paris_time.strftime('%Y-%m-%d %H:%M:%S %Z%z')

# Example of converting the time for bus 244
departure_time_utc = '2024-09-23T22:30:00.000Z'
paris_time = convert_to_paris_time(departure_time_utc)

print(f"Converted Paris time: {paris_time}")


Converted Paris time: 2024-09-24 00:30:00 CEST+0200
